In [84]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf 
import tensorflow_io as tfio
import numpy as np
import time
import soundfile

In [85]:
wav, sample_rate = soundfile.read('/media/kursat/Data/projects/Tansel Bey/tflite_files/signal_TEST/shovel/4.5Hz-35.wav') #file path
new_file_name = 'shovel-4.5Hz-35.wav'
new_folder = '/media/kursat/Data/projects/Tansel Bey/tflite_files/signal_TEST/deneme'
os.chdir(new_folder)
soundfile.write(new_file_name, wav, sample_rate, subtype='PCM_16')#save as PCM 16 for tensorflow.audio.decode.wav
os.chdir('/media/kursat/Data/projects/Tansel Bey/tflite_files') #code file's path


In [86]:
def load_wav(filename):
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    wav = tf.squeeze(wav, axis=1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

In [87]:
#load tflite model
interpreter = tf.lite.Interpreter(model_path = '/media/kursat/Data/projects/Tansel Bey/tflite_files/new_model.tflite')
interpreter.allocate_tensors()

In [88]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
print(input_shape) #input shape 1, 100, 1

[  1 100   1]


In [89]:
yeni_dosya_yolu = new_folder + '/' + new_file_name
print(yeni_dosya_yolu)
#Sound files should be PCM 16 format and we should take first 100 steps of fft output
wav = load_wav(yeni_dosya_yolu)#Read PCM 16 sound file
wav = tf.compat.v1.to_complex64(wav) #converted complex64 format for fft
spectrogram = tf.signal.fft(wav) #fft
spectrogram = tf.abs(spectrogram)
spectrogram = tf.expand_dims(spectrogram, axis=1) #fft
spectrogram = spectrogram[0:100, :] #First 100 steps was taken because there are differeces between classes in first 100 steps
print(type(spectrogram))
spectrogram = np.expand_dims(spectrogram, axis=0)#Using np.expand_dims function for using set_tensor function
input_data = spectrogram

/media/kursat/Data/projects/Tansel Bey/tflite_files/signal_TEST/deneme/shovel-4.5Hz-35.wav
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [90]:
interpreter.set_tensor(input_details[0]['index'], input_data)

In [91]:
time_before= time.time()
interpreter.invoke()
time_after = time.time()
total_tflite_time = time_after - time_before
print("Total prediction time for tflite model is: ", total_tflite_time)

Total prediction time for tflite model is:  0.00039124488830566406


In [92]:
output_data_tflite = interpreter.get_tensor(output_details[0]['index']) #Predicted 0:Shovel, 1:Pickaxe, 2:Walk 
print("The prediction is: ", output_data_tflite, "0:Shovel, 1:Pickaxe, 2:Walk")
output_data_tflite = output_data_tflite.flatten()
print("Prediction: ", np.argmax(output_data_tflite), ' Probability: ', max(output_data_tflite)) #Index was taken that has max values and probability

The prediction is:  [[9.9867672e-01 1.3232960e-03 3.5689052e-14]] 0:Shovel, 1:Pickaxe, 2:Walk
Prediction:  0  Probability:  0.9986767
